**Задание**
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [1]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re


In [23]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten, add
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [6]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [7]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [8]:
y_train = df_train['class'].values
y_val = df_val['class'].values

**Свёрточные архитектуры**

In [9]:
# model1 = Sequential()

# model1.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30))
# model1.add(Conv1D(30, 3, activation='relu'))
# model1.add(GlobalMaxPool1D())
# model1.add(Flatten())
# model1.add(Dense(10, activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(1, activation='sigmoid'))

# model1.summary()

In [10]:
# model1.compile(
#     optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# early_stopping=EarlyStopping(monitor='val_loss')  


# history = model1.fit(X_train, y_train,
#                     batch_size=512,
#                     epochs=10,
#                     verbose=1,
#                     validation_split=0.1,
#                     callbacks=[early_stopping])

In [12]:
# score = model1.evaluate(X_valid, y_val, batch_size=512, verbose=1)
# print('\n')
# print('Test score:', score[0])
# print('Test accuracy:', score[1])

**Архитектуры с RNN**

In [13]:
model2 = Sequential()

model2.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model2.add(SimpleRNN(64))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 30)            7743240   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                6080      
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 7,753,545
Trainable params: 7,753,545
Non-trainable params: 0
_________________________________________________________________


In [14]:
model2.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model2.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 43s 83ms/step - loss: 0.6180 - accuracy: 0.6333 - val_loss: 0.4951 - val_accuracy: 0.7557


In [15]:
score = model2.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 6ms/step - loss: 0.5064 - accuracy: 0.7422


Test score: 0.5063796043395996
Test accuracy: 0.7422298789024353


**Архитектуры с LSTM**

In [16]:
model3 = Sequential()

model3.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model3.add(LSTM(64, recurrent_dropout=0.2))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(1, activation='sigmoid'))

model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24320     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 7,771,785
Trainable params: 7,771,785
Non-trainable params: 0
_________________________________________________________________


In [17]:
model3.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model3.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 73s 216ms/step - loss: 0.6095 - accuracy: 0.6468 - val_loss: 0.4912 - val_accuracy: 0.7575


In [18]:
score = model2.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 6ms/step - loss: 0.5064 - accuracy: 0.7422


Test score: 0.5063796043395996
Test accuracy: 0.7422298789024353


In [19]:
model4 = Sequential()

model4.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
model4.add(LSTM(64, recurrent_dropout=0.2))
model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(1, activation='sigmoid'))

model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24320     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 7,771,785
Trainable params: 7,771,785
Non-trainable params: 0
_________________________________________________________________


In [20]:
model4.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model4.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 73s 218ms/step - loss: 0.6079 - accuracy: 0.6522 - val_loss: 0.4923 - val_accuracy: 0.7565


In [21]:
score = model4.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 27ms/step - loss: 0.5108 - accuracy: 0.7391


Test score: 0.5107614994049072
Test accuracy: 0.7391438484191895


In [25]:
inputs = Input(shape=(X_train.shape[1],))

x = Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)(inputs)

x2 = LSTM(30, return_sequences=True)(x)
x = add([x, x2])
x2 = LSTM(30, return_sequences=True)(x)
x = add([x, x2])
x2 = LSTM(30, return_sequences=True)(x)
x = add([x, x2])


x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)



model5 = Model(inputs=inputs, outputs=x)
model5.summary()
model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 27)]         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 27, 30)       7743240     input_3[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, 27, 30)       7320        embedding_5[0][0]                
__________________________________________________________________________________________________
add_3 (Add)                     (None, 27, 30)       0           embedding_5[0][0]                
                                                                 lstm_6[0][0]               

In [26]:
history = model5.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 139s 401ms/step - loss: 0.6033 - accuracy: 0.6550 - val_loss: 0.4881 - val_accuracy: 0.7584


In [27]:
score = model5.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 45ms/step - loss: 0.5128 - accuracy: 0.7419


Test score: 0.5127558708190918
Test accuracy: 0.7418771982192993


**GRU**

In [30]:
model6 = Sequential()

model6.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30))
model6.add(GRU(64, recurrent_dropout=0.2))
model6.add(Dense(64, activation='relu'))
model6.add(Dropout(0.5))
model6.add(Dense(1, activation='sigmoid'))


model6.summary()

model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
gru (GRU)                    (None, 64)                18432     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 7,765,897
Trainable params: 7,765,897
Non-trainable params: 0
_________________________________________________________________


In [31]:
history = model6.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 70s 208ms/step - loss: 0.6051 - accuracy: 0.6574 - val_loss: 0.4913 - val_accuracy: 0.7570


In [32]:
score = model6.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 24ms/step - loss: 0.5140 - accuracy: 0.7431


Test score: 0.5140463709831238
Test accuracy: 0.7431116104125977


**CNN -> RNN**

In [37]:
model7 = Sequential()

model7.add(Embedding(input_dim=word_count, input_length=training_length, output_dim=30))
model7.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding="same"))
model7.add(LSTM(64, return_sequences=True))
model7.add(Flatten())
model7.add(Dense(64, activation='relu'))
model7.add(Dropout(0.5))
model7.add(Dense(1, activation='sigmoid'))

model7.summary()
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 64)            5824      
_________________________________________________________________
lstm_10 (LSTM)               (None, 27, 64)            33024     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1728)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                110656    
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [ ]:
history = model7.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10


In [ ]:
score = model7.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])